Tutorial 4 Trajectory inference

In [ ]:
library(CytoTRACE)
library(monocle3)
library(ggplot2)
library(reshape2)
library(ggpubr)
#library(ggthemes)
library(RColorBrewer)

In [ ]:
y <- read.csv("MACA_bonemarrow_10x_expr.txt", row.names =1)
pheno <- read.table("Cell_Cluster_ID.txt", row.names =1, comment.char = "", check.names = F, header = T)
pheno <- cbind.data.frame(pheno, Somite = gsub(".*\\.", "", rownames(pheno)))
pheno$Cluster <- paste0("C", pheno$Cluster)
y2 <- y[,rownames(pheno)]
clusters<- pheno$Cluster_Name
names(clusters) <- rownames(pheno)

In [ ]:
coord <- read.csv("tsne-coordinates.csv", row.names =1)
coord <- coord[rownames(pheno),]
#Monocle
genes <- cbind.data.frame(rownames(y2))
rownames(genes) <- rownames(y2)
cds <- new_cell_data_set(data.matrix(y2),
                         cell_metadata = data.frame(pheno),
                         gene_metadata = genes)
cds <- preprocess_cds(cds, num_dim = 100)
cds2 <- reduce_dimension(cds)
cds2@int_colData$reducedDims$UMAP[,1] <- coord[,1]
cds2@int_colData$reducedDims$UMAP[,2] <- coord[,2]
#You can adjust resolution here which will change how the branches look in the graphs
cds2 <- cluster_cells(cds2,cluster_method = "louvain")
#cds2 <- cluster_cells(cds2,cluster_method = "leiden", resolution = 0.01)
cds2@clusters$UMAP$partitions[which(cds2@clusters$UMAP$partitions != "1")] <- "1"
cds2 <- learn_graph(cds2, use_partition = F)
cds_subset <- order_cells(cds2)
cds2@clusters$UMAP$clusters <- paste0("C", cds2@clusters$UMAP$clusters)

plot1 = plot_cells(cds2, color_cells_by='Cluster_Name', label_branch_points = F, label_roots = F, label_leaves = F, cell_size = 1, group_cells_by = "cluster", labels=F)
cds2@colData$Cluster <- cds2@clusters$UMAP$clusters
plot2 = plot_cells(cds2, color_cells_by='Cluster', label_branch_points = F, label_roots = F, label_leaves = F, cell_size = 1, group_cells_by = "cluster", labels=F)
plot3 = plot_cells(cds2, color_cells_by='Somite', label_branch_points = F, label_roots = F, label_leaves = F, cell_size = 1, group_cells_by = "cluster", labels=F)
plot4 = plot_cells(cds_subset,
           color_cells_by = "pseudotime",
           label_cell_groups=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE,
           graph_label_size=1.5,
           cell_size = 1)

cols1 <- unique(pheno$Color)
names(cols1) <- unique(pheno$Cluster_Name)
pdf("Monocle_plot1.pdf", width = 5, height = 5, useDingbats = FALSE)
plot1+
scale_color_manual("legend", values = cols1)+
theme_Publication()+
theme(legend.position = "none",
panel.grid.major = element_blank(), 
panel.grid.minor = element_blank())
dev.off()

pdf("Monocle_plot2_legend.pdf", width = 5, height = 5, useDingbats = FALSE)
plot2+
theme_Publication()+
theme(legend.position = "none",
panel.grid.major = element_blank(), 
panel.grid.minor = element_blank())
dev.off()

cols3 <- c("#fed700", "#fe0000", "#00fe01", "#070089")
names(cols3) <- unique(pheno$Somite)
pdf("Monocle_plot3.pdf", width = 5, height = 5, useDingbats = FALSE)
plot3+
scale_color_manual("legend", values = cols3)+
theme_Publication()+
theme(legend.position = "none",
panel.grid.major = element_blank(), 
panel.grid.minor = element_blank())
dev.off()

pdf("Monocle_plot4.pdf", width = 5, height = 5, useDingbats = FALSE)
plot4+
scale_colour_gradientn(colours = brewer.pal(11, "Spectral"))+
theme_Publication()+
theme(legend.position = "none",
panel.grid.major = element_blank(), 
panel.grid.minor = element_blank())
dev.off()
